In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving 1342-0.txt to 1342-0.txt


In [ ]:
file = open("/content/1342-0.txt", "r", encoding = "utf8")

# store file in list
lines = []
for i in file:
    lines.append(i)

# Convert list to string
data = ""
for i in lines:
  data = ' '. join(lines) 

#replace unnecessary stuff with space
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','')  #new line, carriage return, unicode character --> replace by space

#remove unnecessary spaces 
data = data.split()
data = ' '.join(data)
data[:1000]

'The Project Gutenberg eBook of Pride and Prejudice, by Jane Austen This eBook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this eBook or online at www.gutenberg.org. If you are not located in the United States, you will have to check the laws of the country where you are located before using this eBook. Title: Pride and Prejudice Author: Jane Austen Release Date: June, 1998 [eBook #1342] [Most recently updated: August 23, 2021] Language: English Character set encoding: UTF-8 Produced by: Anonymous Volunteers and David Widger *** START OF THE PROJECT GUTENBERG EBOOK PRIDE AND PREJUDICE *** THERE IS AN ILLUSTRATED EDITION OF THIS TITLE WHICH MAY VIEWED AT EBOOK [# 42671 ] cover Pride and Prejudice By Jane Austen CONTENTS Chapter 1 Chapter 2 Chapter 3 Chapter 4 Chapter 5 Chapter 6 Chapt

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

# saving the tokenizer for predict function
pickle.dump(tokenizer, open('token.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:15]

[1, 176, 158, 916, 3, 321, 4, 1171, 30, 72, 2534, 41, 916, 23, 21]

In [ ]:

vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

7030


In [ ]:
sequences = []

for i in range(3, len(sequence_data)):
    words = sequence_data[i-3:i+1]
    sequences.append(words)
    
print("The Length of sequences are: ", len(sequences))
sequences = np.array(sequences)
sequences[:10]

The Length of sequences are:  125306


array([[   1,  176,  158,  916],
       [ 176,  158,  916,    3],
       [ 158,  916,    3,  321],
       [ 916,    3,  321,    4],
       [   3,  321,    4, 1171],
       [ 321,    4, 1171,   30],
       [   4, 1171,   30,   72],
       [1171,   30,   72, 2534],
       [  30,   72, 2534,   41],
       [  72, 2534,   41,  916]])

In [ ]:
X = []
y = []

for i in sequences:
    X.append(i[0:3])
    y.append(i[3])
    
X = np.array(X)
y = np.array(y)

In [ ]:
print("Data: ", X[:10])
print("Response: ", y[:10])

Data:  [[   1  176  158]
 [ 176  158  916]
 [ 158  916    3]
 [ 916    3  321]
 [   3  321    4]
 [ 321    4 1171]
 [   4 1171   30]
 [1171   30   72]
 [  30   72 2534]
 [  72 2534   41]]
Response:  [ 916    3  321    4 1171   30   72 2534   41  916]


In [ ]:
y = to_categorical(y, num_classes=vocab_size)
y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=3))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("next_words.h5", monitor='loss', verbose=1, save_best_only=True)
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001))
model.fit(X, y, epochs=70, batch_size=64, callbacks=[checkpoint])

Epoch 1/70
1955/1958 [============================>.] - ETA: 0s - loss: 6.2258
Epoch 1: loss improved from inf to 6.22538, saving model to next_words.h5
1958/1958 [==============================] - 37s 15ms/step - loss: 6.2254
Epoch 2/70
1957/1958 [============================>.] - ETA: 0s - loss: 5.6264
Epoch 2: loss improved from 6.22538 to 5.62618, saving model to next_words.h5
1958/1958 [==============================] - 29s 15ms/step - loss: 5.6262
Epoch 3/70
1955/1958 [============================>.] - ETA: 0s - loss: 5.3022
Epoch 3: loss improved from 5.62618 to 5.30237, saving model to next_words.h5
1958/1958 [==============================] - 30s 15ms/step - loss: 5.3024
Epoch 4/70
1957/1958 [============================>.] - ETA: 0s - loss: 5.0712
Epoch 4: loss improved from 5.30237 to 5.07117, saving model to next_words.h5
1958/1958 [==============================] - 30s 15ms/step - loss: 5.0712
Epoch 5/70
1956/1958 [============================>.] - ETA: 0s - loss: 4.8657
E

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle

# Load the model and tokenizer
model = load_model('next_words.h5')
tokenizer = pickle.load(open('token.pkl', 'rb'))

def Predict_Next_Words(model, tokenizer, text):

  sequence = tokenizer.texts_to_sequences([text])
  sequence = np.array(sequence)
  preds = np.argmax(model.predict(sequence))
  predicted_word = ""
  
  for key, value in tokenizer.word_index.items():
      if value == preds:
          predicted_word = key
          break
  
  print(predicted_word)
  return predicted_word

In [ ]:
while(True):
  text = input("Enter your line: ")
  
  if text == "0":
      print("Execution completed.....")
      break
  
  else:
      try:
          text = text.split(" ")
          text = text[-3:]
          print(text)
        
          Predict_Next_Words(model, tokenizer, text)
          
      except Exception as e:
        print("Error occurred: ",e)
        continue

Enter your line: a truth universally
['a', 'truth', 'universally']
acknowledged
Enter your line: Bennet made no
['Bennet', 'made', 'no']
answer
Enter your line: I see no
['I', 'see', 'no']
occasion
Enter your line: i love
['i', 'love']
it
Enter your line: the movie
['the', 'movie']
party
Enter your line: comes into the
['comes', 'into', 'the']
neighbourhood
Enter your line: when he comes into the
['comes', 'into', 'the']
neighbourhood
Enter your line: 0
Execution completed.....
